In [1]:
from re_rl.dataset_generator import DatasetGenerator

In [2]:
generator = DatasetGenerator()
dataset = generator.generate_dataset(
    task_types=["linear", "quadratic"],
    languages=["ru", "en"],
    detail_levels=[1, 2, 3],
    tasks_per_type=100
)
generator.save_dataset(dataset)

Датасет сохранен в datasets/dataset_20250331_003911.json
